In [34]:
import numpy as np 
import matplotlib.pyplot as plt 
from spgl1 import spg_bpdn
from util.fns import *
from util.util import *
from util.dists import *
from util.activations import *

In [21]:
m = 1000
N = 10000
d = 5

ratio_train = 0.75
q = 2
normalize = False

In [29]:
X = make_X(d, m, dist=uniform)
W = make_W(d, N, q, dist=normal)
A = make_A(X, W, active=fourier)


In [30]:
y = (1+np.linalg.norm(X, axis=-1))**(-0.5)
A_train = A[:int(m*ratio_train), :]
A_test = A[int(m*ratio_train):, :]
y_train = y[:int(m*ratio_train)]
y_test = y[int(m*ratio_train):]

In [47]:
# L1
c_spgl1, _, _, _ = spg_bpdn(A_train, y_train, sigma=0.1)

Linesearch failed with error 1. Damping max BB scaling to 10000.0
Linesearch failed with error 1. Damping max BB scaling to 1000.0
Linesearch failed with error 1. Damping max BB scaling to 100.0
Linesearch failed with error 1. Damping max BB scaling to 10.0
Linesearch failed with error 1. Damping max BB scaling to 1.0
Linesearch failed with error 1. Damping max BB scaling to 0.1
Linesearch failed with error 1. Damping max BB scaling to 0.01
Linesearch failed with error 1. Damping max BB scaling to 0.001
Linesearch failed with error 1. Damping max BB scaling to 0.0001


In [48]:
ind_spgl1, = c_spgl1.nonzero()
print('\nPercentage of nonzero entries in the coefficient using SPGL1', 100*ind_spgl1.shape[0]/(2*N))


Percentage of nonzero entries in the coefficient using SPGL1 100.0


In [56]:
y_train_spgl1 = np.matmul(A_train, c_spgl1)
error_train_spgl1 = np.linalg.norm(y_train - y_train_spgl1)/ (m*ratio_train)
y_test_spgl1 = np.matmul(A_test, c_spgl1)
error_test_spgl1 = np.linalg.norm(y_test - y_test_spgl1)/(m*(1-ratio_train))

In [58]:
print(error_train_spgl1)
print(error_test_spgl1)

0.00014667722993347924
0.00024757928064329815


In [54]:
c_l2 = np.linalg.pinv(A_train)@y_train

In [60]:
y_train_l2 = np.matmul(A_train, c_l2)
error_train_l2 = np.linalg.norm(y_train - y_train_l2)/ (m*ratio_train)
y_test_l2 = np.matmul(A_test, c_l2)
error_test_l2 = np.linalg.norm(y_test - y_test_l2)/(m*(1-ratio_train))

In [61]:
print(error_train_l2)
print(error_test_l2)

1.296374699354099e-09
0.005362244852238697


In [66]:
# weights for weighted min-norm solution
norm = np.linalg.norm(W, axis=1)**2
weight = np.concatenate((norm, norm), axis=-1)
D = np.diag(1.0/weight**4)
c_weighted = D@np.linalg.pinv(A_train@D)@y_train

In [67]:
y_train_weighted = np.matmul(A_train, c_weighted)
error_train_weighted = np.linalg.norm(y_train - y_train_weighted)/ (m*ratio_train)
y_test_weighted = np.matmul(A_test, c_weighted)
error_test_weighted = np.linalg.norm(y_test - y_test_weighted)/(m*(1-ratio_train))

In [68]:
print(error_train_weighted)
print(error_test_weighted)

0.00016742487403485537
0.00032334397830077483
